<a href="https://colab.research.google.com/github/cleysonl/ML_Bootcamp_CLL/blob/master/Advanced_ML_SM_DT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydotplus

In [0]:
import numpy as np
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/dipanjanS/appliedml_workshop_dhs_av_2019/master/Module%2008%20-%20%20Advanced%20ML%20-%20Supervised%20Models/data/Wine_Quality_Data.csv')

### **Encode Target Variable**

In [0]:
data['color'] = data.color.replace('white', 0).replace('red', 1).astype(np.int)

In [4]:
data.sample(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color
698,9.4,0.615,0.28,3.20,0.087,18.0,72.0,1.00010,3.31,0.53,9.7,5,1
5132,6.6,0.220,0.30,14.70,0.045,50.0,136.0,0.99704,3.14,0.37,10.6,6,0
3809,7.4,0.240,0.26,1.60,0.058,53.0,150.0,0.99360,3.18,0.50,9.9,7,0
1934,6.3,0.230,0.30,1.80,0.033,16.0,91.0,0.99060,3.28,0.40,11.8,6,0
4294,5.7,0.230,0.28,9.65,0.025,26.0,121.0,0.99250,3.28,0.38,11.3,6,0
2167,6.1,0.260,0.51,2.20,0.050,61.0,154.0,0.99290,3.08,0.60,9.8,6,0
4210,6.5,0.300,0.39,7.80,0.038,61.0,219.0,0.99590,3.19,0.50,9.4,5,0
2645,6.5,0.190,0.10,1.30,0.046,23.0,107.0,0.99370,3.29,0.45,10.0,5,0
6221,6.5,0.500,0.22,4.10,0.036,35.0,131.0,0.99020,3.26,0.55,13.0,7,0
2209,6.5,0.360,0.28,3.20,0.037,29.0,119.0,0.99080,3.25,0.65,12.4,8,0


In [0]:
feature_cols = [x for x in data.columns if x not in 'color']

# use StratifiedShuffleSplit to split into train and test sets that are stratified
from sklearn.model_selection import StratifiedShuffleSplit

#split the data into two parts with 1000 points in the test data
#this creates a generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=1000, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['color']))

#Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'color']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'color']

In [0]:
X_train.shape, X_test_shape